### Homework 4

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#### Data Preparation

* Check if there are missing values  present in the features.
* If there are missing values:
  * For categorical features, replace them with 'NA'.
  * For numerical features, replace with 0.0.

In [2]:
# Get the dataset. We will use the lead scoring dataset Bank Marketing dataset.

data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"

In [3]:
!wget $data -O data-hw3.csv

--2025-10-22 10:03:46--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘data-hw3.csv’

data-hw3.csv        100%[===================>]  78.98K  --.-KB/s    in 0.002s  

2025-10-22 10:03:46 (50.3 MB/s) - ‘data-hw3.csv’ saved [80876/80876]



In [4]:
df = pd.read_csv("data-hw3.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
numerical = [ 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score' ]

In [7]:
categorical = [ 'lead_source', 'industry', 'employment_status', 'location' ] 

In [8]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
# Fill missing values in categorical variables with 'NA'

df['lead_source'] = df.lead_source.fillna('NA')
df['industry'] = df.industry.fillna('NA')
df['employment_status'] = df.employment_status.fillna('NA')
df['location'] = df.location.fillna('NA')

In [10]:
# Fill missing values in numerical variables with 0.0

df['annual_income'] = df.annual_income.fillna(0.0)

In [11]:
# Check again for missing values in all variables

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

#### Dataset Split

Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use train_test_split function for that with random_state=1.

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [14]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [15]:
876+293+293, len(df), 0.2*len(df), 0.6*len(df)

(1462, 1462, 292.40000000000003, 877.1999999999999)

In [16]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
# Check that 'converted' is not in the features set
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

#### Question 1: ROC AUC Feature Importance

ROC AUC could also be used to evaluate feature importance.

* For each numerical variable, use it as score (aka prediction) and compute the AUC with the y variable as ground truth.
* Use the training dataset for that.

In [20]:
from sklearn.metrics import roc_auc_score

In [21]:
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [22]:
roc_auc_score(y_train, df_train['lead_score'])

0.6144993577250176

In [23]:
roc_auc_score(y_train, df_train['number_of_courses_viewed'])

0.7635680590007088

In [24]:
roc_auc_score(y_train, df_train['interaction_count'])

0.738270176293409

In [25]:
roc_auc_score(y_train, df_train['annual_income'])

0.5519578313253012

In [27]:
# Which numerical variable has the highest AUC? 'number_of_courses_viewed'

#### Question 2: Training the model

Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

logisticRegression(solver='liblinear', C=1.0, max_iter=1000)